In [ ]:
import conda_scripts
import pandas as pd
import geopandas as gpd
import datetime
import os

In [ ]:
bas = "SRP"

In [ ]:
wl = conda_scripts.wiski.wiski.get_gw_stations_wellmeta_in_basin(basins = [f'{bas}*'])

In [ ]:
wl


In [ ]:
stats = wl.dropna(subset='station_no')
stats = stats.dropna(subset='station_longitude')
stats = stats.loc[stats.station_longitude<0]

In [ ]:
stats.loc[:, 'station_latitude'] = pd.to_numeric(stats.station_latitude, errors='coerce')
stats.loc[:, 'station_longitude'] = pd.to_numeric(stats.station_longitude, errors='coerce')
stats = stats.astype({'station_latitude': float, 'station_longitude': float}, errors='ignore')

stats = gpd.GeoDataFrame(stats, geometry=gpd.points_from_xy(stats.station_longitude, stats.station_latitude),
                         crs=4326).to_crs(2226)

In [ ]:
stats.explore()

In [ ]:
wshed = conda_scripts.load_datasets.load_watersheds.load(f'{bas}')

In [ ]:
stats = gpd.clip(stats, wshed)

In [ ]:
stats

In [ ]:
k = conda_scripts.wiski.wiski.get_kiwis()

In [ ]:
# load parameters for basin

locs = [f'{bas}*','ADR*', 'PLP*', 'PRM*',  'UPC*' ,'UPE*' ,'WRM*']

son = [k.get_timeseries_list(station_name=bas, ts_name='01.Fin*,DayMeanEdit*,02.Final*',
                             parametertype_name='Groundwater_elevation',
                             return_fields=['station_name', 'station_no', 'station_id', 'ts_id', 'ts_name',
                                            'parametertype_id', 'parametertype_name', 'coverage']

                             )

       for bas in locs]

In [ ]:
pd.concat(son).query("station_name=='PET0173'")

In [ ]:
assert pd.concat(son).ts_name.nunique()>1

In [ ]:
pars = pd.concat(son)
print(f"there are {pars.ts_name.nunique()} parameter types\n\nThey are {pars.ts_name.unique()}")
pars = pars.loc[pd.to_datetime(pars.loc[:,'from']).notnull()]
assert pars.ts_name.nunique()>1, f"there are only {pars.ts_name.nunique()} parameter types\n\nThey are {pars.ts_name.unique()}"

In [ ]:
parsmissing = pars.loc[~pars.station_no.isin(stats.station_no)]
pars = pars.loc[pars.station_no.isin(stats.station_no)]
assert pars.ts_name.nunique()>1, f"there are only {pars.ts_name.nunique()} parameter types\n\nThey are {pars.ts_name.unique()}"

In [ ]:
print(f"from parmissing:\n\nthere are {parsmissing.ts_name.nunique()} parameter types\n\nThey are {parsmissing.ts_name.unique()}\n\n{parsmissing.ts_name.value_counts()}")

print(f"{parsmissing.station_name.apply(lambda x: x[:3]).value_counts()}")
parsmissing.head()

In [ ]:
df = []
for name, group in pars.groupby('ts_id'):
    ts = k.get_timeseries_values(ts_id=name,
                                 **{'from': datetime.datetime(1969, 12, 1)},
                                 return_fields=['Value', 'Timestamp', 'Quality Code'])
    if ts.shape[0] > 0:
        ts.loc[:, 'ts_id'] = name

        df.extend([ts])

tsall = pd.concat(df)

In [ ]:
tsall.index = conda_scripts.wiski.wiski.tz_fix(tsall)

In [ ]:
tsall.shape

In [ ]:
tsall_res = tsall.groupby(['ts_id', "Quality Code",  pd.Grouper(level =0, freq = "1D"), ]).mean().reset_index()

In [ ]:
tsall_res.groupby('ts_id').count().sort_values('Value', ascending = False)

In [ ]:
tsall_res.to_csv(os.path.join('..','waterlevel',f'{bas}_wl.csv'))
pars.to_csv(os.path.join('..','waterlevel',f'{bas}_pars.csv'))
stats.to_csv(os.path.join('..','waterlevel',f'{bas}_stats.csv'))

In [ ]:
os.getcwd()

In [ ]:
tsall.groupby('ts_id').count().sort_values('Value', ascending = False)

In [ ]:
tsall_res.query("ts_id=='59228010'").plot.scatter(x = 'Timestamp', y = 'Value', c = "Quality Code")

In [ ]:
tsall.query("ts_id=='59228010'")

In [ ]:
pars.query("ts_id=='59228010'")